In [1]:
import arcpy, pandas as pd
from arcpy import env
from arcpy.sa import *
from otherfunctions import folders_exist

In [ ]:
# Paths to input datasets
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
tam_out_dir = r"Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM"
tc_ds = root_folder + "\\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"
bands_gee = ["pr", "pet", "ro"] # band names in GEE - for comparison with GEE TerraClimate dataset
tc_vars = ["ppt", "pet", "q"] # variable names according to TerraClimate

# Set arcpy environment variables
env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")
# env.cellSize = "MINOF" # Avoided to prevent huge files
env.cellSize = out_geotiff + "\\ppt_2023_1.tif" # Use TerraClimate resolution as reference for cell size
env.workspace = r"Z:\PhD_Datasets&Analysis\_ProcessingCache"
env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984") # WGS 1984 (4326)

In [3]:
# Get the current environment's spatial reference
spatial_ref = env.outputCoordinateSystem

# Check if a spatial reference is set
if spatial_ref:
    print(f"Spatial Reference Name: {spatial_ref.name}")
    print(f"Spatial Reference WKID: {spatial_ref.factoryCode}")
else:
    print("No spatial reference is set in the current environment.")

Spatial Reference Name: GCS_WGS_1984
Spatial Reference WKID: 4326


In [ ]:
######################################################
### Starting values for the water balance model - T&M
######################################################

# Initial variables
years = range(1958, 2023 + 1) # Years to process. This line can be used to execute this code for specific years in multiple runs.
months = range(1, 12 + 1)

In [ ]:
# Create folders for other variables of tam model
#wyield_dir = tam_out_dir + '\\wyield' # Calculated using bflow
wyield_dir = tam_out_dir + '\\wyield2' # Calculated using bflow2
folders_exist([wyield_dir])

# Folder with baseflow rasters resulting from the model
#bflow_dir = tam_out_dir + '\\bflow' # Calculated at the daily level using daily recession constants (k) and aggregated at the monthly level
bflow_dir = tam_out_dir + '\\bflow2' # Calculated at the monthly level using monthly recession constants (k)

In [ ]:
print('\n############################################################')
print('\t\tINITIAL VARIABLES')
print('\tPeriod to be executed: ' + str(years[0]) + '-' + str(years[-1]))
print('############################################################')

for year in years:

    print("\n**Executing calculation for " + str(year) + "**")

    for month in months:

        print("\n\t*Executing calculation for month " + str(month) + "*\n")

        print("\tLoading baseflow and runoff files.....")

        ro_name = tc_vars[2] + "_" + str(year) + "_" + str(month)
        ro = Raster(out_geotiff + "\\" + ro_name + ".tif") # Read runoff raster of the month

        bflow = Float(Raster(bflow_dir + "\\bflow_" + str(year) + "_" + str(month) + ".tif")) # Read baseflow raster of the month

        print("\tCalculating water yield......")
        wyield = ro + bflow # Calculate water yield (runoff + baseflow)
        wyield.save(wyield_dir + "\\wyield_" + str(year) + "_" + str(month) + ".tif")

arcpy.CheckInExtension("spatial")

# Clear the workspace environment
arcpy.ClearEnvironment("workspace")

print("\nDONE!!")